In [1]:
from typing import List, Tuple
from datetime import datetime, timedelta
import numpy.typing as npt
import numpy as np
import os
import cv2
import pickle
from tqdm import tqdm
import itertools

from utils.compose_factory import compose_factory
from utils.data_dirs import data_dirs
from utils.get_videos import get_videos

In [2]:
raw_dir, processed_dir, folder, EXPERIMENT_DATA, suffix = data_dirs(False)
print(raw_dir)
print(processed_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

/Users/chanwutk/Documents/apperception/data/v1.0-mini
/Users/chanwutk/Documents/apperception/data/v1.0-mini
v1.0-mini
/work/apperception/data/raw/scenic/experiment_data



In [3]:
fps = 20

In [4]:
interpolate = False

In [5]:
partition_dir = os.path.join(processed_dir, 'partition')

In [6]:
compose = compose_factory(fps, raw_dir, processed_dir, interpolate)

In [ ]:
for l in os.listdir(partition_dir):
    d = os.path.join(partition_dir, l)
    with open(os.path.join(d, 'sample_data.pickle'), "rb") as f:
        sample_data = pickle.load(f)
    videos = get_videos(sample_data)
    
    
    output = {}
    for name in videos:
        video = videos[name]
        frames, filename, start = compose(l, name, video)
        output[name] = {
            "filename": filename,
            "start": start,
            "columns": [
                "cameraId",
                "frameId",
                "frameNum",
                "filename",
                "cameraTranslation",
                "cameraRotation",
                "cameraIntrinsic",
                "egoTranslation",
                "egoRotation",
                "timestamp",
                "cameraHeading",
                "egoHeading",
            ],
            "frames": video
        }
        print("done", filename)
    base = os.path.join(processed_dir, 'videos', l)
    with open(os.path.join(base, 'frames.pickle'), "wb") as f:
        pickle.dump(output, f)

Reading scene: scene-0103-CAM_FRONT_LEFT


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 388/388 [00:02<00:00, 139.95it/s]


Writing scene (/Users/chanwutk/Documents/apperception/data/v1.0-mini/videos/boston-seaport/scene-0103-CAM_FRONT_LEFT-2018-8-1-12-26-43-504799.mp4):


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 388/388 [00:02<00:00, 131.43it/s]


done scene-0103-CAM_FRONT_LEFT-2018-8-1-12-26-43-504799.mp4
Reading scene: scene-0103-CAM_FRONT


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 396/396 [00:03<00:00, 110.32it/s]


done scene-0655-CAM_BACK_LEFT-2018-8-27-8-51-32-147405.mp4
Reading scene: scene-0757-CAM_FRONT_LEFT


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 397/397 [00:03<00:00, 120.24it/s]


Writing scene (/Users/chanwutk/Documents/apperception/data/v1.0-mini/videos/boston-seaport/scene-0757-CAM_FRONT_LEFT-2018-8-30-12-25-8-254799.mp4):


 74%|████████████████████████████████████████████████████████████████████████████████████████████████▎                                  | 292/397 [00:03<00:00, 113.41it/s]